*1. Generating Synthetic Supernova Data*

In [ ]:
import numpy as np
import cv2
from sklearn.model_selection import train_test_split

# Function to generate synthetic supernova data
def generate_synthetic_supernova_data(num_samples=1000, img_size=64):
    X = []
    y = []

    for _ in range(num_samples):
        # Create "supernova" images with a white circle in the center
        img_supernova = np.zeros((img_size, img_size), dtype=np.uint8 )
        cv2.circle(img_supernova, (img_size//2, img_size//2), 5, (255), -1)  # Bright spot in center
        X.append(img_supernova)
        y.append(1)  # Label 1 for supernova

        # Create "no supernova" images (random noise)
        img_no_supernova = np.random.randint(0, 255, (img_size, img_size), dtype=np.uint8)
        X.append(img_no_supernova)
        y.append(0)  # Label 0 for no supernova

    X = np.array(X).reshape(-1, img_size, img_size, 1) / 255.0  # Normalize
    y = np.array(y)
    return X, y

# Generate synthetic data
X, y = generate_synthetic_supernova_data(num_samples=1000)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


*2. Model Architecture: Convolutional Neural Network (CNN)*

In [16]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import os
import cv2
from sklearn.model_selection import train_test_split

# Generate more robust synthetic data
def generate_data(num_samples=1000, img_size=64):
    X, y = [], []

    for _ in range(num_samples):
        # Supernova images with noise
        img_supernova = np.random.randint(0, 50, (img_size, img_size), dtype=np.uint8)
        cv2.circle(img_supernova, (img_size//2, img_size//2), np.random.randint(5, 10), (255), -1)
        X.append(img_supernova)
        y.append(1)

        # No supernova images
        img_no_supernova = np.random.randint(0, 255, (img_size, img_size), dtype=np.uint8)
        X.append(img_no_supernova)
        y.append(0)

    X = np.array(X).reshape(-1, img_size, img_size, 1) / 255.0
    y = np.array(y)
    return X, y
# Generate data
X, y = generate_data(num_samples=1000)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Define the CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])


# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, validation_split=0.2, epochs=20, batch_size=32, verbose=1)

# 4. Evaluate the Model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# 5. Save the Model
model.save('supernova_detector.h5')
print("Model saved as 'supernova_detector.h5'")


Epoch 1/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 6s 76ms/step - accuracy: 0.9293 - loss: 0.1898 - val_accuracy: 1.0000 - val_loss: 6.6000e-08
Epoch 2/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 72ms/step - accuracy: 1.0000 - loss: 4.0621e-08 - val_accuracy: 1.0000 - val_loss: 2.2297e-08
Epoch 3/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 76ms/step - accuracy: 1.0000 - loss: 2.2054e-08 - val_accuracy: 1.0000 - val_loss: 2.1746e-08
Epoch 4/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 68ms/step - accuracy: 1.0000 - loss: 2.4956e-08 - val_accuracy: 1.0000 - val_loss: 2.1667e-08
Epoch 5/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 69ms/step - accuracy: 1.0000 - loss: 2.3088e-08 - val_accuracy: 1.0000 - val_loss: 2.1590e-08
Epoch 6/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 69ms/step - accuracy: 1.0000 - loss: 2.2409e-08 - val_accuracy: 1.0000 - val_loss: 2.1509e-08
Epoch 7/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 70ms/step - accuracy: 1.0000 - loss: 2.0290e-08 - val_accuracy: 1.0000 - val_loss: 2.1421e-08
Epoch 8/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 70ms/step - accura

Test Accuracy: 100.00%
Model saved as 'supernova_detector.h5'


*5. Classifying Uploaded Images*

In [3]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import tensorflow as tf
import numpy as np
from PIL import Image
import io

# Load the trained model 
try:
    model = tf.keras.models.load_model('supernova_detector.h5')
except FileNotFoundError:
    print("Error: Model file 'supernova_detector.h5' not found.")
    raise

# Function to preprocess the uploaded image
# Function to preprocess the uploaded image
def preprocess_image(uploaded_image):
    try:
        # Open the image and convert it to grayscale
        img = Image.open(uploaded_image).convert('L')  # Convert to grayscale
        img = img.resize((64, 64))  # Resize to match model input
        img = np.array(img) / 255.0  # Normalize pixel values
        img = np.expand_dims(img, axis=(0, -1))  # Add batch and channel dimensions
        return img
    except Exception as e:
        print(f"Error processing the image: {e}")
        raise


# Function to classify the uploaded image with debugging
def classify_image(change):
    try:
        # Clear previous output
        clear_output(wait=True)
        display(uploader)  # Re-display the uploader widget
        
        # Check if a file was uploaded
        if not change['new']:
            print("No file uploaded.")
            return

        # Get the uploaded file content
        uploaded_file = change['new'][0]
        img_data = uploaded_file['content']
        
        # Preprocess the image
        # Debugging the preprocessing step
        img = preprocess_image(io.BytesIO(img_data))
        print("Processed Image Shape:", img.shape)
        print("Processed Image Pixel Range:", img.min(), img.max())

        # Visualize the processed image
        import matplotlib.pyplot as plt
        plt.imshow(img[0, :, :, 0], cmap='gray')
        plt.title("Uploaded Image After Preprocessing")
        plt.show()

        # Predict the class
        prediction = model.predict(img)[0][0]
        print(f"Raw Prediction: {prediction:.4f}")  # Display raw prediction
        
        # Classify based on threshold
        threshold = 0.4  # Adjust based on observed raw predictions
        result = "Supernova Detected" if prediction > threshold else "No Supernova Detected"

        print(f"Result: {result}")
    except Exception as e:
        print(f"Error: {e}")

# Create the upload widget
uploader = widgets.FileUpload(
    accept='image/*',  # Accept image files
    multiple=False  # Single file upload
)

# Attach the event handler
uploader.observe(classify_image, names='value')

# Display the uploader widget
print("Upload an image to classify:")
display(uploader)


Upload an image to classify:


FileUpload(value=(), accept='image/*', description='Upload')